# Abstract
Causal inference under Pearl’s framework of structural causal models has been the subject of intense theoretical study, but performant open-source implementations of many of the key algorithms are still lacking or, where they exist, are not widely adopted or maintained. I present a performant implementation of Shpitser’s IDC algorithm for identification of conditional causal effects in semi-Markovian causal models.

# Introduction

Causal inference is a fundamental task in many fields, including epidemiology, economics, and computer science. One popular framework for causal inference is Pearl's structural causal modeling framework, which utilizes directed acyclic graphs (DAGs) and the do-calculus to explicitly model potential causal relationships. This approach allows for a clear and intuitive representation of causal assumptions, while also helping to avoid common biases that can arise with other methodologies. However, a crucial step in this framework is identification: translating a causal question into an abstract statistical estimator that can be estimated using a combination of a model and parametric assumptions.

One important algorithm for identification in the structural causal modeling framework is Shpitser's algorithm for conditional interventional effects. This algorithm is polynomial time and guarantees a result when one is possible. However, existing open-source implementations of this algorithm are often incorrect, slow, or unmaintained. In this paper, we present a novel implementation of Shpitser's algorithm, written in Rust for efficient cross-functional use. Our implementation is designed for ease of use and includes extensive testing routines to ensure performance and correctness.

The paper is organized as follows: in the first section, we provide an overview of the background and motivation behind the algorithm. In the second section, we describe our implementation in detail. In the third section, we provide examples to demonstrate the utility of our implementation. Appendices include code and HC/LO footnotes.

## Contributions

In this work, we present a novel implementation of Shpitser's ID algorithm, including a comprehensive set of graph routines and a heuristic-based simplification algorithm. The implementation is written in Rust and the code is provided in the appendix. Additionally, we have developed Python bindings to facilitate usage of the algorithm. It is important to note that while this implementation is novel, the underlying theoretical work is not a contribution of this paper and is instead a summary of the current state of the field.

# Background

## Structural Causal Models

Structural Causal Models (SCM) provide a framework for modeling the generating process of a distribution. A structural causal model has four components: a set of observed variables $\textbf{V} = {v_1, v_2, ..., v_n}$, a set of unobserved variables $\textbf{U} = {u_1, u_2, ..., u_m}$, a distribution $P(U)$ over unobserved variables, and a set of functions $\textbf{F} = {f_1, f_2...f_n}$ such that each $f_i$ is a function from a subset of $\textbf{U} \cup \textbf{V} \backslash V_i$ to $v_i$. The arguments to $f_i$ are called the parents of $v_i$ or $pa(v_i)$ and they uniquely determine the value of $v_i$. Together, $P(\textbf{U})$ and $\textbf{F}$ induce $P(\textbf{V})$, a distribution over $\textbf{V}$.

We represent the causal assumptions in an SCM via a causal diagram. This is a directed acyclic graph (DAG) where each node corresponds to a variable in the model. A directed edge from $x_i$ to $x_j$ indicates that the value of $x_j$ depends on the value of $x_i$, or $x_i \in pa(x_j)$. To ensure the validity of an SCM, we restrict the set of valid models to those that can be represented by acyclic graphs, to prevent circular definitions between variables [1]. Algebraically, we can interpret a causal diagram as defining a set of conditional independence assumptions. In particular, as the parents of $v_i$ completely determine its value, $v_i$ will be independent of every other variable in the model, conditional on its parents.

For the purposes of this paper, we make the additional assumption that each unobserved variable is the parent of at most two observed variables. This simplifies the graphical representation by replacing each unobserved variable and its outgoing edges with a single undirected edge connecting its two children. This can be interpreted as representing potential confounding between the two variables. An example of such a simplified graph is shown in Figure 1.

[1] This does not restrict the class of distributions that can be represented by an SCM because by the definition of joint probability any joint distribution $P(x_1, x2…x_n)$ can be factorized as $P(x_1 | x_2…x_n)P(x_2 | x_3…x_n)...P(x_n-1 | x_n) P(x_n)$. But, this can be represented as an SCM where $pa(x_i) = \{x_{i+1}, x_{i+2}…x_n\}$ without violating the acyclicity assumption.

**Figure 1. Using Bidirected Edges to Represent Confounding**
![](2023-01-14-13-06-10.png)
*This figure shows an unobserved variable $C$ with two children being replaced with a bidirected, dashed edge. The bidirected edge represents the same information as $C$ did in the first diagram, but it is only possible to represent confounders with bidirected edges if they have exactly two children.*

## Conditional Independence Properties of Causal Diagrams

One of the key properties of causal diagrams is that they can provide insight into the conditional independence properties of a distribution $P(\textbf{U} \cup \textbf{V})$. As previously discussed, a variable $v_i$ is independent of all other variables in the model, conditional on its parents. This concept of a "flow of dependence" along edges connecting a node to its parent is a useful tool for understanding the conditional independence assumptions encoded in a causal diagram.

However, it is important to note that dependence does not flow along every path in the diagram. Paths can be blocked by conditioning on certain variables, which alters the conditional independence properties of the distribution. We can view a path as a collection of vertices, each of which must have one of the following forms:

$$X \leftarrow Y \leftarrow Z$$
$$X \rightarrow Y \rightarrow Z$$
$$X \leftarrow Y \rightarrow Z$$
$$X \rightarrow Y \leftarrow Z$$

In the first three cases, dependence will flow through the vertex as long as we do not condition on $Y$. In this case, we call the vertex unblocked. Conditioning on $Y$ blocks the path and prevents dependence from traveling along it. This means that, as long as this is the only path connecting $X$ and $Y$, the causal diagram is predicting that $X$ and $Z$ will be independent conditional on $Y$.

The third case, known as a collider, is slightly different. The last vertex will be unblocked if and only if we condition on $Y$.

To illustrate this concept, consider the example of college admissions. Without knowledge of whether a student was admitted, learning her SAT scores tells us nothing about her admissions essays. However, if we know that the student was admitted, and we observe that her SAT scores were low, we can infer that her essays must have been good.

A path is considered blocked if any vertex along it is blocked. If, after conditioning on a set of variables $\textbf{Z}$, there are no unblocked paths connecting two variables $v_i$ and $v_j$, then the model predicts that $v_i$ and $v_j$ are independent conditional on $\textbf{Z}$. In this case, we say that $v_i$ and $v_j$ are d-separated by $\textbf{Z}$. This concept of d-separation provides a means to connect the ideas of statistical association in the joint distribution with properties of the graph, and allows us to use graphical models of causal relationships to make testable predictions about datasets.

**Figure 2. Examples of D-Separation**

![](2023-01-14-13-52-24.png)

*The figure shows an example of a causal graphical model. Nodes represent random variables, and edges represent direct causal influence.*

For example, in figure 2 we can make the following predictions about distribution $P(\textbf{V})$ based on the causal diagram:

- $A \perp\!\!\!\!\perp C$ but not $A \perp\!\!\!\!\perp C | D$
- $A \perp\!\!\!\!\perp B | C$ but not $A \perp\!\!\!\!\perp B | C, D$
- $B \perp\!\!\!\!\perp D | C$ but not $B \perp\!\!\!\!\perp D$

## Graph Surgery and Causal Effects

In causal inference, a fundamental task is to measure the causal effect of one set of variables, $X$, on another, $Y$. To operationalize this task within the Structural Causal Model (SCM) framework, we utilize the concept of interventions. Specifically, in the SCM, the values of $X$ are determined by its parents, $pa(X)$. However, we are interested in understanding the effect of altering the value of $X$ through intervention, rather than allowing it to be set by its natural causes. To model this intervention process, we perform "graph surgery" by cutting all incoming edges into $X$, and examining the resulting interventional distribution. This allows us to isolate any remaining conditional effect of $X$ on $Y$, which can then be interpreted as a causal effect.

To gain further insight into this process, we can consider two types of paths that may connect $X$ and $Y$: backdoor paths and frontdoor paths. Backdoor paths are those that connect to $X$ through its parents, while frontdoor paths connect through its children. These paths represent different mechanisms through which $X$ may affect $Y$; frontdoor paths indicate direct causal effects, while backdoor paths indicate indirect effects through other variables (i.e. confounding). By performing graph surgery and cutting all backdoor paths, we effectively isolate any causal effects flowing through frontdoor paths.


## The Do-Calculus

In the Structural Causal Model (SCM) framework, causal queries are represented by the use of the "do-operator." The do-operator is indicated by its placement behind the conditioning bar in a probability expression, and it represents a modification of the underlying generating process. To illustrate, for sets of variables $X$, $Y$, and $Z$, the expression $P(Y | do(X), Z)$ represents the conditional probability $P(Y | X, Z)$ in the model that results from performing graph surgery on $X$. This represents a hypothetical scenario in which $X$ is fixed through intervention, rather than being determined by "nature." A "do-expression" is any probability expression containing the do-operator.

Both the average treatment effect (ATE) and the conditional average treatment effect (CATE) can be expressed in terms of do-expressions. The ATE is given by:

$ATE = E(P(y | do(x = 1))) - E(P(y | do(x = 0)))$

while the CATE is given by:

$CATE = E(P(y | z, do(x = 1))) - E(P(y | z, do(x = 0)))$

The manipulation of do-expressions is facilitated by Pearl's "do-calculus", which references properties of the model in which the do-expression is evaluated. The do-calculus provides conditions under which do-operators can be removed from an expression or replaced with conditioning. This is advantageous because it allows us to transform do-expressions, which cannot be estimated from the data, into traditional statistical estimands (e.g., $P(Y | X, Z)$), which can be estimated using parametric assumptions.

$
\textbf{Rule 1 } \text{Insertion/deletion of observations:}\\
P(Y | do(X), Z, W) = P(Y | do(X), W) \\
\text{if } (Y \perp\!\!\!\!\perp Z | X, W)_{G_{\overline{X}}}
$

$
\textbf{Rule 2 } \text{Action/observation interchange:}\\
P(Y | do(X), do(Z), W) = P(Y | do(X), Z, W) \\
\text{if } (Y \perp\!\!\!\!\perp Z | X, W)_{G_{\overline{X} \underline{Z}}}
$

$
\textbf{Rule 3 } \text{Insertion/deletion of actions:}\\
P(Y | do(X), do(Z), W) = P(Y | do(X), W) \\
\text{if } (Y \perp\!\!\!\!\perp Z | X, W)_{G_{\overline{X} \overline{Z(W)}}}
$

The task of removing the do-operators from a do-expression and turning it into a statistical estimand is called "identification." It is a crucial step in the process of causal inference in the SCM framework. General procedures for identification allow us to express causal queries in the rich language of do-expressions and then algorithmically translate these queries into statistical estimands that can be directly estimated from the data. Additionally, because identification does not make any parametric assumptions about the data, it allows us to decouple our causal assumptions from our parametric assumptions during modeling.

## The Four Stages of Causal Inference

Structured causal effect estimation is a four-step process consisting of modeling, identification, estimation, and robustness checks.

1. Modeling: This step involves identifying plausible causal relationships in the form of a directed acyclic graph (DAG). The DAG serves as a representation of the causal structure of the system being studied.

2. Identification: Once the causal structure is represented in the form of a DAG, the next step is to translate a causal query into an abstract estimator in the context of these assumptions. This process is known as identification, and it is a crucial step in the process of causal inference.

3. Estimation: After the abstract estimator has been derived, the next step is to add parametric assumptions and pick a specific model to represent the abstract estimator. This model is then fitted to the data, and an estimate of the causal effect is extracted.

4. Robustness checks: The final step is to assess the robustness of the analysis to the specifics of the assumptions made in the modeling process. This is done by perturbing the assumptions of the modeling process and determining the sensitivity of the analysis to these perturbations. These robustness checks are important for ensuring the validity and generalizability of the causal estimates.

It's important to note that the modeling, identification, estimation and robustness checks are interrelated, and that making progress in one step may require revisiting the others.

## The Problem of Identification

It is important to note that not every causal estimand can be unambiguously estimated in every model. Identification, or the process of determining the causal estimand from the joint distribution, may fail when it is impossible to unambiguously determine $P(Y | do(X))$ from $P$. Identification is always possible in a graph with no confounding, but in some cases it is not possible because it is not possible to disentangle the effect from possible confounding. For example, in a model with a bidirected edge between variables, it may be impossible to identify $P(Y | do(X))$ because any observed effect could be explained by confounding along the bidirected edge, and there is no way to control for this.

**Figure 4. The Bow Graph**

![](2023-01-14-14-22-06.png)

*The bow graph is the simplest example of a graph with a non-identifiable interventional distribution.*

The rules of the do-calculus provide complete conditions for identification of causal estimands; if it is possible to convert the causal estimand to a statistical estimand, then it is possible using the do-calculus. However, manually applying the rules of the do-calculus to derive an abstract estimator is time-consuming and requires the researcher to make reference to complex properties of the causal graph, which is a task humans are ill-suited to, especially as the complexity of the graph grows. In the general case, it is too complex a task to be performed by hand.

Thus, it is necessary to find algorithms by which identification can be performed automatically. This is a nontrivial problem; naive approaches often have exponential time complexity in the size of the graph, and the best algorithms, while polynomial time, rest on complex graph theory.

Some researchers have simplified this problem by sidestepping direct application of the do-calculus and instead using the concept of adjustment sets. These methods rely on finding a set of variables which, when controlled for, block all backdoor-paths from the intervention set to the outcome set. However, these approaches face two distinct challenges. First, naive approaches to adjustment set search, such as those implemented in Knupple (2010), Breitling (2010), and Sharma (2020) rely on enumerating the set of all possible adjustment sets and then checking whether each successfully blocks all backdoor paths. This is an issue because the number of possible adjustment sets grows exponentially in the number of nodes in the graph. This makes these approaches infeasible for even fairly small graphs.

More efficient algorithms exist for adjustment set search, such as the one proposed in Textor and Liskiewicz (2011), which works in polynomial time. However, the drawback of these algorithms is that although every valid adjustment set corresponds to a valid statistical estimator for the query, not every valid statistical estimator can be represented as an adjustment set. Because these algorithms are only searching among adjustment sets, they fail in cases where other classes of estimator are needed.

To illustrate, in the famous front-door model, the interventional distribution $P(Y | do(X))$ is identified by the simple statistical estimator $P(Y | do(X)) = \sum_z P(Z | X) \sum_x P(Y | Z, X) P(X)$. This estimator works by breaking the problem into two subproblems: first identifying the interventional distributions $P(M | do(X))$ and $P(Y | do(M))$ and then combining these to identify $P(Y | do(X))$. Each subproblem can be solved using an adjustment set operator, but there is no adjustment set estimator which identifies $P(Y | do(X))$.

## C-components

Front-door estimation is an interesting technique, but it also illustrates a larger concept in causal inference known as c-component factorization. C-components, or connected components, are subsets of the graph that are connected via bidirectional (confounding) edges. Two nodes, $A$ and $B$, in a causal graph are in the same c-component if and only if there is a path from $A$ to $B$ that only follows bidirectional edges. It has been shown that any causal graph can be uniquely partitioned into a minimal set of c-components (Tian, 2002).

More importantly, Tian (2002) showed that for any c-component $C$ in a graph $G$, the interventional distribution $P(C | do(V \backslash C))$ is identifiable, where $V$ denotes the set of nodes in $G$ and $\backslash$ represents set difference. This means that the problem of identifying $P(Y | do(X))$ can be broken down into a set of subproblems, one for each c-component in the graph $G \backslash X$. This is the first critical insight that enables efficient identification and is known as c-component factorization.

C-components also allow for the introduction of more complex graphical structures, such as the c-tree. A $Y$-rooted c-tree is a graph $G$ that contains both directed and bidirected edges, where the entire graph is a single c-component and every node has at most one child. Additionally, there is a single node $Y$ that has no children. If there are multiple nodes with no children, these nodes are referred to as the root-set of $G$, and the graph is called a c-forest.

**Figure 4. A C-Tree and  C-Forest**

![](2023-01-14-15-01-52.png)

*This figure shows a c-tree (left) and a c-forest (right). Note that both are c-components because they are completely connected by bidirectional edges. In addition, in both, no node has more than one child. However, in the c-tree, C is the only node without children. This makes this a $C$-rooted c-tree. On the other hand, the graph on the right has two nodes without children: $C$ and $G$. This makes it a $\{C, G\}$-rooted c-forest.*

Furthermore, by utilizing c-forests, the concept of a hedge can be defined. Hedges are graphical structures that prevent the identification of interventional distributions. In a causal graphical model with graph $G$, if an interventional distribution $P(Y | do(X))$ is sought for two sets of random variables $X$ and $Y$, and if there exist two subgraphs $F$ and $F'$ such that the intersection of $F$ and $X$ is non-empty, but the intersection of $F'$ and $X$ is empty, and if $F'$ is a subgraph of $F$, and both form $Z$-rooted c-forests, where $Z$ is a subset of $Y$, the interventional distribution $P(Y | do(X))$ will be identifiable if and only if such $F$, $F'$, and $Z$ do not exist.

In other words, if two nested c-forests are found, where at least some elements from $Y$ are in their root set, and the smaller one does not contain any elements from $X$ but the bigger one does, the interventional distribution is not identifiable.

## Shpitser's Algorithm

Shpitser’s ID algorithm exploits the properties of c-components to identify interventional distributions. It is polynomial time and can always derive an estimator if one exists.
 
ID works by breaking up the identification problem into subproblems. Considering the subgraph $G’ = G \setminus X$, we find the c-components $\{s_i\}$ of $G’$. If there is more than one, then we can identify $P(Y | do(X))$ by finding the interventional distribution $P(s_i | do(G \ s_i))$ for each $s_i$. Because of c-component factorization, we know that all such terms will be identifiable if and only if $P(Y | do(X))$ is identifiable. We can then take the product of all such terms to get a joint interventional distribution over the nodes in $G’$. By marginalizing all variables other than those in $X$ and $Y$, we can identify the interventional distribution.
 
After repeated application of the above step, we will be in a position where $G’$ consists of a single c-component. There are three possible ways in which $X$ could relate to $G’$. (1) $X$ and $G’$ could together form a single c-component. In this case, we have discovered a hedge, proving the interventional distribution is not identifiable. (2) There could be no bidirectional edges connecting elements of $G’$ to elements of $X$. In this case, there are no backdoor paths from $X$ to $G’$ and we can identify the interventional distribution by conditioning on $X$. (3) It could be that some elements $Z \subset X$ are connected via bidirectional edges to elements in $G’$, while others $W \subset X$ are not. In this case, we can condition on $W$ and make a recursive call to the algorithm, attempting to identify the interventional distribution $P(G’ | do(Z))$.
 
The algorithm as described so far can only deal with queries of the form $P(Y | do(X))$, but often we will wish to calculate conditional interventional effects of the form $P(Y | do(X), Z)$. Thankfully, queries of this form can be mapped to queries without a conditional term with only a minimal transformation.

## Pre-existing Implementations

A number of open source implementations of the ID algorithm existed prior to my work. Dowhy is a popular python library developed by Microsoft Research, but its implementation is too slow. Researchers working on the dowhy project explicitly asked me to create a faster implementation and integrate it with their library. There is also an implementation in Clojure called Whittemore which is quite fast, but because Clojure is a JVM language it is not possible to integrate this implementation with Python, R, or Javascript code. I based my implementation on Whittemore. Ananke is a causal inference library developed by Shpitser’s team, and it has an implementation of a related algorithm, but Ananke has not seen widespread adoption, and it is unmaintained. FInally, the Javascript/R tool Daggity has an implementation of adjustment set operators, which suffer from the drawbacks discussed above.

# Approach & Technical Challenges

## Graph Representation and Subroutines
As a prerequisite to the rest of the work I did for this project, I implemented a set of graph routines to capture key operations that need to be performed during the algorithm. There are two kinds of graphs that I implemented: directed and bidirected graphs. The most important operations for the directed graph are to find ancestors of a given element or determine $\text{d-separation}$. The most important operation for the bidirected graph is to find connected components.

The tricky thing about these routines is that, as the algorithm runs, they are repeatedly called on slightly modified versions of the original graph. So, for example, a common question we might ask while executing the algorithm is: if we removed node $X$, would nodes $Y$ and $W$ still be in the same connected component? Or, if we intervened on $G$, would $A$ still be an ancestor of $B$? The most naive implementations of subroutines like these would be to copy the data from the original graph, modify the copy to reflect the operation, and then run the subroutine.

Instead, I implemented a set of routines that allows small modifications to be performed to the original graphs in such a way that a new data structure is created which is logically distinct inside the program but which uses most of the same memory. This reduces the memory footprint of the algorithm as well as reducing time spent copying data.

## Identification
The implementation of identification itself is based on the description in Shpitser’s dissertation and the Whittemore implementation.

## Simplification
Simplification is an important step in computational identification because the ID algorithm tends to return symbolic expressions that are much more complex than they need to be. It is important to simplify the results for a couple of reasons. Simpler expressions are easier for humans to understand. They make it easier to test the program because they reduce the number of “correct” answers to an identification problem. And, maybe most importantly, simpler expressions are easier to evaluate numerically. It is both computationally cheaper and more accurate to simplify an expression before applying numerical estimation techniques.

Whittemore, which I have borrowed from extensively, uses a heuristic-based simplification algorithm with just a couple of rules. My original plan was to just use this, but as I was implementing it I found a quirk in the way it simplifies certain quotients that I am 90% sure is a bug, although I haven’t gotten around to proving that it’s broken yet.

Tikka and Karvanen (2017) describe a simplification routine that uses properties of the graphical model to find ways to reduce conditional terms. I was originally optimistic about this approach as it has nice guarantees on the degree of simplification. However, this approach has a time complexity that gorws significantly faster than exponential in the number of variables, and so would not prove practical for real problems.

Instead of relying on these solutions, I developed my own heuritsitc simplification algorithm.

## Python Bindings
While the focus of the Rust implementation is correctness and performance, the focus of the Python bindings is on elegance and usability. I used Python's magic methods to create an API for specifying and querying causal models that is extremely simple and readable. I created a website with documentation for the python library, and I have published it on PyPI.

# Examples

In this section, I show how my code can be used to identify interventional distributions in two toy examples.